<a href="https://colab.research.google.com/github/chicago-cdac/2022-bpi-clinic/blob/vincent-liu-bpi/IDOT_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferencing from Illinois Department of Transportation Traffic Stop Study

## Set up

Data notebook: https://idot.illinois.gov/Assets/uploads/files/Transportation-System/Pamphlets-&-Brochures/Safety/2012DBFileFormat.pdf

In [ ]:
!pip install altair 
!pip install altair_saver

In [425]:
import pandas as pd
import altair as alt
from datetime import datetime


In [ ]:
def add_footer_to_plot(chart, footer_text):
	  return alt.concat(chart).properties(title=alt.TitleParams(footer_text,
							 baseline='bottom',
							 orient='bottom',
     					 anchor='start',
     					 fontSize=12, 
     					 fontWeight=200, dy=20)).configure_legend(labelLimit= 0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks/BPI/"

Mounted at /content/drive


## Loading Data and Cleaning

In [ ]:
idot21 = pd.read_csv(path+"IDOT_2021.csv")
idot21.head(5)

,DATESTOP,TIMESTOP,DURATION,OFFNAME,OFFBDGE,CITY_I,STATE,VEHMAKE,VEHYEAR,YRBIRTH,...,DOGALERT,DOGALERTSRCH,DOGALERTSRCHCONTRA,DOGDRUG,DOGPARA,DOGALC,DOGWEAP,DOGSTOLPROP,DOGOTHER,DOGDRAMT
0,1/1/21,0:33,5,FIDEL LEGORRETA,5902.0,CHICAGO,IL,CHEVROLET,2017.0,1993,...,0,0,0,0,0,0,0,0,0,0
1,1/1/21,1:50,4,VICTOR PEREZ,7383.0,CHICAGO,IL,FORD,2012.0,1957,...,0,0,0,0,0,0,0,0,0,0
2,1/1/21,8:50,4,STEPHANIE ORTIGARA,18302.0,CHICAGO,IL,FORD,2007.0,1967,...,0,0,0,0,0,0,0,0,0,0
3,1/1/21,12:41,6,JASON ARROYO,14502.0,CHICAGO,IL,BMW,1998.0,1990,...,0,0,0,0,0,0,0,0,0,0
4,1/1/21,13:51,5,MONTY OWENS,11975.0,CHICAGO,IL,TOYOTA,2002.0,1945,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def cleaning(data):
    """
    Cleaning the IDOT (Illinois Department of Transportation) data.
    The function does the following: 
        * Convert data/time related variables to their specific types
        * Convert drivers' races and sexs to a string
        * Filter out unnecessary columns
    Return: 
        a cleaned IDOT dataset (as a pandas dataframe)  
    """
    def convert_DATESTOP(datestr):
        try:
            return datetime.strptime(datestr, '%m/%d/%Y').date()
        except:
            return datetime.strptime(datestr, '%m/%d/%y').date()

    def convert_TIMESTOP(timestr):
        try:
            if timestr[1] == ':':
                timestr = '0' + timestr
            if len(timestr) == 8:
                return datetime.strptime(timestr, '%H:%M:%S').time()
            return datetime.strptime(timestr, '%H:%M').time()
        except:
            if timestr[0:2] == '0:':
                timestr = '12' + timestr[1:]
            return datetime.strptime(timestr, '%I:%M:%S %p').time()

    data['DATESTOP'] = data['DATESTOP'].apply(lambda x: convert_DATESTOP(x))
    data['TIMESTOP'] = data['TIMESTOP'].apply(lambda x: convert_TIMESTOP(x))

 #   data = data[['DATESTOP', 'TIMESTOP', 'DURATION', 
 #           'YRBIRTH', 'DRSEX', 'DRRACE', ""
 #           ]]
    data.loc[:,'YEARSTOP'] = data.loc[:,'DATESTOP'].apply(lambda x: x.year)
    data.loc[:,'DRAGE'] = data.apply(lambda x: x['YEARSTOP'] - x['YRBIRTH'], axis=1)
    data.loc[:,'DRRACE'] = data.loc[:,'DRRACE'].astype(str)
    data.loc[:,'DRSEX'] = data.loc[:,'DRSEX'].astype(str)

    return data

In [ ]:
idot21 = cleaning(idot21)

In [ ]:
idot21 = idot21[idot21.CITY_I == "CHICAGO"]  # ONLY in Chicago
idot21.loc[:,'MONTH'] = idot21.loc[:,'DATESTOP'].apply(lambda x: x.month) # subtracting month from a date type

idot21.dtypes #looking at the types

DATESTOP               object
TIMESTOP               object
DURATION                int64
OFFNAME                object
OFFBDGE               float64
CITY_I                 object
STATE                  object
VEHMAKE                object
VEHYEAR               float64
YRBIRTH                 int64
DRSEX                  object
DRRACE                 object
REASSTOP              float64
TYPEMOV               float64
RESSTOP                 int64
BEAT_I                  int64
VEHCONSREQ              int64
VEHCONSGIV              int64
VEHSRCHCOND             int64
VEHSRCHCONDBY           int64
VEHCONTRA               int64
VEHDRUGS                int64
VEHPARA                 int64
VEHALC                  int64
VEHWEAP                 int64
VEHSTOLPROP             int64
VEHOTHER                int64
VEHDRAMT                int64
DRCONSREQ               int64
DRCONSGIV               int64
DRVSRCHCOND             int64
DRVSRCHCONDBY           int64
PASSCONSREQ             int64
PASSCONSGI

## Driver Demographics and Incident Locations

In [ ]:
drivers = idot21[['DATESTOP', 'TIMESTOP', 'DURATION', 'MONTH',
            'YRBIRTH', 'DRSEX', 'DRAGE', 'DRRACE',
            'REASSTOP', 'RESSTOP','BEAT_I']]

for r in range(-3,0,1):
  drivers.iloc[:,r] = drivers.iloc[:, r].astype(str) # converting reason to stop, result of stop, and beats to categorical

drivers.dtypes

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


DATESTOP    object
TIMESTOP    object
DURATION     int64
MONTH        int64
YRBIRTH      int64
DRSEX       object
DRAGE        int64
DRRACE      object
REASSTOP    object
RESSTOP     object
BEAT_I      object
dtype: object

### Part I: By Month

In [ ]:
month_count = drivers.groupby("MONTH")["DATESTOP"].count().reset_index()
month_count.columns = ["MONTH","COUNT"]

for i in range(month_count.shape[0]):
    month = str(month_count.MONTH[i])
    month = datetime.strptime(month, "%m").strftime("%b")
    month_count.loc[i,"MONTH_NAME"] = month
month_count

,MONTH,COUNT,MONTH_NAME
0,1,32924,Jan
1,2,25789,Feb
2,3,31356,Mar
3,4,25089,Apr
4,5,26983,May
5,6,22958,Jun
6,7,21929,Jul
7,8,17307,Aug
8,9,19388,Sep
9,10,21421,Oct


In [ ]:
from pandas.api.types import CategoricalDtype
#cat_type = CategoricalDtype(categories= month_count.MONTH_NAME,
  #                           ordered=True)
month_count.iloc[:,0] = month_count.iloc[:,0].astype(str)
month_count.iloc[:,-1] = month_count.iloc[:,-1].astype(str)

In [ ]:
month_count.dtypes

MONTH         object
COUNT          int64
MONTH_NAME    object
dtype: object

In [ ]:
list(month_count.MONTH_NAME.values)

['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']

In [ ]:
alt.Chart(month_count, title="Number of Traffic Stops in Chicago by Month in 2021").mark_line(color='steelblue').encode(  
		    	  x = alt.X('MONTH:T', title='', axis=alt.Axis(format='%b', grid =False)),  
	        	y = alt.Y('COUNT', title='', axis=alt.Axis(domain = False, ticks = False, grid = True, 
                                                     gridColor = "#D1D4D6", gridWidth = 1)),
         ).configure_axis(
      grid=False
  ).configure_view(
      strokeWidth=0
  ).properties(width=500, height=250,
          #     title=alt.TitleParams(     # This works but at the cost of sacrificing the title
     #   [' ', 'Source: 2021 Illinois Department ofTransportation', ' Traffic and Pedestrian Stop Study (IDOT-ITPSS)'],
    #    baseline='bottom',
     #   orient='bottom',
    #    anchor='end',
     #   fontWeight='normal',
    #    fontSize=10
  #  )
               )

#chart = add_footer_to_plot(chart, "Source: Illinois Department ofTransportation Traffic and Pedestrian Stop Study")
#chart

alt.Chart(...)

In [ ]:
!npm install vega-lite vega-cli canvas
!pip install selenium
!pip install urllib3
!pip install altair_saver

In [ ]:
month_count.dtypes

MONTH          int64
COUNT          int64
MONTH_NAME    object
dtype: object

### Part II: By Drivers' Race and Sex

* Race 

In [ ]:
race_count = drivers.groupby(["DRRACE"])["MONTH"].count().reset_index()
race_count.columns = ['DRRACE', 'COUNT']
race_count["PERCENT"] = round(race_count.COUNT/race_count.COUNT.sum(), 4)*100
race_count

,DRRACE,COUNT,PERCENT
0,American Indian or Alaska Native,901,0.31
1,Asian,6632,2.29
2,Black or African American,188854,65.18
3,Hispanic or Latino,66899,23.09
4,Native Hawaiian or Other Pacific Islander,587,0.20
5,White,25884,8.93


In [ ]:
alt.Chart(race_count, title = "Number of Drivers Stopped by Officers in Traffic by Race").mark_bar().encode(
      x= alt.X('DRRACE', title = "" , sort = "-y", axis=alt.Axis(labelAngle=-45, domain = False, ticks = False)),
      y= alt.Y('COUNT', title = "", axis=alt.Axis(tickCount = 7, ticks = False, domain=False))
  ).configure_axis(
      grid=False
  ).configure_view(
      strokeWidth=0
  ).properties(width = 300, height = 300)



alt.Chart(...)

* Race and Sex

In [ ]:
race_sex_count = drivers[drivers.DRSEX != '3'] # remove unknown sex (n = 278)
race_sex_count = race_sex_count.groupby(["DRRACE", "DRSEX"])["MONTH"].count().reset_index()
race_sex_count.columns = ['DRRACE', 'DRSEX', 'COUNT']
race_sex_count

,DRRACE,DRSEX,COUNT
0,1.0,1,17019
1,1.0,2,8849
2,2.0,1,122498
3,2.0,2,66154
4,3.0,1,719
5,3.0,2,180
6,4.0,1,48257
7,4.0,2,18590
8,5.0,1,5071
9,5.0,2,1555


In [ ]:
race_name = {'1.0':"White", '2.0':"Black or African American", 
             '3.0': "American Indian or Alaska Native", 
             '4.0': "Hispanic or Latino",
             '5.0': "Asian",
             '6.0': "Native Hawaiian or Other Pacific Islander"}

sex_name = {"1": "Male", "2": "Female"}

race_sex_count.iloc[:,0] = race_sex_count['DRRACE'].map(race_name) 
race_sex_count.iloc[:,1] = race_sex_count['DRSEX'].map(sex_name) 

In [ ]:
drivers.loc[:,"DRRACE"] = drivers['DRRACE'].map(race_name) 
drivers.loc[:,"DRSEX"] = drivers['DRSEX'].map(sex_name) 

In [ ]:
race_sex_count = race_sex_count.iloc[0:-2,:]
race_sex_count

,DRRACE,DRSEX,COUNT
0,White,Male,17019
1,White,Female,8849
2,Black or African American,Male,122498
3,Black or African American,Female,66154
4,American Indian or Alaska Native,Male,719
5,American Indian or Alaska Native,Female,180
6,Hispanic or Latino,Male,48257
7,Hispanic or Latino,Female,18590
8,Asian,Male,5071
9,Asian,Female,1555


In [ ]:
alt.Chart(race_sex_count, title="Proportion of Drivers in Each Race and Sex Group In Traffic Stops").mark_bar().encode(
    y= alt.Y('DRRACE', sort= ['White'], title='', axis=alt.Axis(ticks=False, domain =False)),
    x=alt.X('COUNT', stack="normalize", title='', axis=alt.Axis(format='%', ticks=False, domain =False)),
    color= alt.Color('DRSEX', sort=["blue", 'red'], legend=alt.Legend(orient='none', title = '', direction = "horizontal", legendX = 200, legendY = -20)),
    text=alt.Text('COUNT')
).configure_axis(
      grid=False
  ).configure_view(
      strokeWidth=0
  ).properties(width = 500, height = 200)

#add_footer_to_plot(chart3, "Source: Illinois Department ofTransportation Traffic and Pedestrian Stop Study")

alt.Chart(...)

### Part III: By Beat

* Beat Only

In [ ]:
def reformat_beats(df):
  """
  Add digit '0' to beats that are three characters in length  

  ### Example
  Eg. For a beat that is currently labelled '972', the function will do the trick to make it "0972".
  """
  df = df.reset_index().iloc[:,1:]
  nrow = df.shape[0]
  for row in range(nrow):
    beat_lab = df.loc[row, "BEAT_I"]
    if len(beat_lab) == 3:
      beat_lab = "0" + beat_lab
      df.loc[row, "BEAT_I"] = beat_lab
  return df

In [ ]:
drivers.reset_index()

In [ ]:
drivers = reformat_beats(drivers)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
beat = drivers.groupby(["BEAT_I"])["MONTH"].count().reset_index()
beat.columns = ["BEAT_I", "COUNT"]
beat = beat.sort_values(by = 'COUNT', ascending = False)
beat.head(5)

,BEAT_I,COUNT
129,1112,5424
140,1133,4741
130,1113,4469
132,1115,4367
133,1121,4142


In [ ]:
beat20 = beat.head(20)

In [ ]:
bars = alt.Chart(beat20, title = "Top 20 Police Beats in Chicago with the Most Traffic Stops in 2021").mark_bar().encode(
    x= alt.X('COUNT', axis=alt.Axis(title='', domain = False, ticks=False, tickCount=5, grid=False)),
    y=alt.Y('BEAT_I', sort= '-x', 
            axis=alt.Axis(title='Beat Number', 
            titleFontSize = 14, titleFontWeight = 600, titleAngle = 0,
            titleY = -5, titleX = -10, titlePadding= 10,ticks=False, domain=False))
    )#.configure_axis(domain=False)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='COUNT:Q'
)

(bars + text).properties(height=500, width = 600)


alt.LayerChart(...)

* Beat and Race

In [ ]:
beat_race = drivers.groupby(["BEAT_I", "DRRACE"])["MONTH"].count().reset_index()
beat_race.columns = ["BEAT_I", "DRRACE", "COUNT"]
beat_race.head(6) 

,BEAT_I,DRRACE,COUNT
0,0111,American Indian or Alaska Native,8
1,0111,Asian,45
2,0111,Black or African American,463
3,0111,Hispanic or Latino,140
4,0111,Native Hawaiian or Other Pacific Islander,5
5,0111,White,145


In [ ]:
race_simp = {"White": "White", "Black or African American": "Black or African American",
             "Hispanic or Latino": "Hispanic or Latino", "Asian": "Other",
             "Native Hawaiian or Other Pacific Islander": "Other",
             "American Indian or Alaska Native":"Other"}
drivers.loc[:,"DRRACE_NEW"] = drivers['DRRACE'].map(race_simp) 

In [ ]:
beat_race_10 = drivers.groupby(["BEAT_I", "DRRACE_NEW"])["MONTH"].count().reset_index()
beat_race_10.columns = ["BEAT_I", "DRRACE_NEW", "COUNT"]
beat_race_10 = beat_race_10[beat_race_10.BEAT_I.isin(beat.head(10).BEAT_I.unique())]
beat_race_10.head(4) 

,BEAT_I,DRRACE_NEW,COUNT
311,0725,Black or African American,3132
312,0725,Hispanic or Latino,290
313,0725,Other,12
314,0725,White,20


In [ ]:
alt.Chart(beat_race_10, title="Racial Disparities in Traffic Stops for top-10 Highest-traffic Stop Police Beats").mark_bar().encode(
    y= alt.Y('BEAT_I', title='', sort= '-x', axis=alt.Axis(title='Beat Number', 
            titleFontSize = 12, titleFontWeight = 600, titleAngle = 0,
            titleY = -5, titleX = -10, titlePadding= 10,ticks=False, domain=False)),
    x=alt.X('COUNT', stack="normalize", title='', axis=alt.Axis(format='%', ticks=False, domain=False)),
    color= alt.Color('DRRACE_NEW', legend=alt.Legend(orient='none', title = '', direction = "horizontal", legendX = 100, legendY = -20),
                     scale=alt.Scale(scheme='category10')),
    text=alt.Text('COUNT')
).configure_axis(
      grid=False
  ).configure_view(
      strokeWidth=0
  ).properties(width = 600, height = 300)

alt.Chart(...)

In [ ]:
drivers.to_csv(path+"drivers_subset.csv")

## Consent Searches and Contrabands

### Consent Searches

The legal discusion on police consents: probable cause will not be needed should a consent for search is given by the automobile driver. That is, an officer is legitimized to search a car even if he does not have sufficient evidence about the concealing of contrabands in the car amount to the level of a probable cause, which is a degree far beyond pure guesses. 

In [ ]:
idot21.loc[:,'DRRACE'] = idot21['DRRACE'].map(race_name) 
idot21.loc[:, 'DRSEX'] = idot21['DRSEX'].map(sex_name) 
idot21.loc[:,"DRRACE_NEW"] = idot21['DRRACE'].map(race_simp) 

In [ ]:
idot21.columns

Index(['DATESTOP', 'TIMESTOP', 'DURATION', 'OFFNAME', 'OFFBDGE', 'CITY_I',
       'STATE', 'VEHMAKE', 'VEHYEAR', 'YRBIRTH', 'DRSEX', 'DRRACE', 'REASSTOP',
       'TYPEMOV', 'RESSTOP', 'BEAT_I', 'VEHCONSREQ', 'VEHCONSGIV',
       'VEHSRCHCOND', 'VEHSRCHCONDBY', 'VEHCONTRA', 'VEHDRUGS', 'VEHPARA',
       'VEHALC', 'VEHWEAP', 'VEHSTOLPROP', 'VEHOTHER', 'VEHDRAMT', 'DRCONSREQ',
       'DRCONSGIV', 'DRVSRCHCOND', 'DRVSRCHCONDBY', 'PASSCONSREQ',
       'PASSCONSGIV', 'PASSSRCHCOND', 'PASSSRCHCONDBY', 'PASSDRVCONTRA',
       'PASSDRVDRUGS', 'PASSDRVPARA', 'PASSDRVALC', 'PASSDRVWEAP',
       'PASSDRVSTOLPROP', 'PASSDRVOTHER', 'PASSDRVDRAMT', 'DOGPERFSNIFF',
       'DOGALERT', 'DOGALERTSRCH', 'DOGALERTSRCHCONTRA', 'DOGDRUG', 'DOGPARA',
       'DOGALC', 'DOGWEAP', 'DOGSTOLPROP', 'DOGOTHER', 'DOGDRAMT', 'YEARSTOP',
       'DRAGE', 'MONTH', 'DRRACE_NEW'],
      dtype='object')

In [ ]:
search = idot21[["MONTH", "DRSEX", "DRRACE", "DRRACE_NEW"] + [col for col in idot21.columns if 'VEH' in col]]
search.columns

Index(['MONTH', 'DRSEX', 'DRRACE', 'DRRACE_NEW', 'VEHMAKE', 'VEHYEAR',
       'VEHCONSREQ', 'VEHCONSGIV', 'VEHSRCHCOND', 'VEHSRCHCONDBY', 'VEHCONTRA',
       'VEHDRUGS', 'VEHPARA', 'VEHALC', 'VEHWEAP', 'VEHSTOLPROP', 'VEHOTHER',
       'VEHDRAMT'],
      dtype='object')

In [ ]:
search.head(5)

,MONTH,DRSEX,DRRACE,DRRACE_NEW,VEHMAKE,VEHYEAR,VEHCONSREQ,VEHCONSGIV,VEHSRCHCOND,VEHSRCHCONDBY,VEHCONTRA,VEHDRUGS,VEHPARA,VEHALC,VEHWEAP,VEHSTOLPROP,VEHOTHER,VEHDRAMT
0,1,Male,Hispanic or Latino,Hispanic or Latino,CHEVROLET,2017.0,2,0,2,0,0,0,0,0,0,0,0,0
1,1,Male,Black or African American,Black or African American,FORD,2012.0,2,0,2,0,0,0,0,0,0,0,0,0
2,1,Male,Black or African American,Black or African American,FORD,2007.0,2,0,2,0,0,0,0,0,0,0,0,0
3,1,Male,Black or African American,Black or African American,BMW,1998.0,2,0,2,0,0,0,0,0,0,0,0,0
4,1,Female,Black or African American,Black or African American,TOYOTA,2002.0,2,0,2,0,0,0,0,0,0,0,0,0


In [ ]:
search.dtypes

MONTH              int64
DRSEX             object
DRRACE            object
DRRACE_NEW        object
VEHMAKE           object
VEHYEAR          float64
VEHCONSREQ         int64
VEHCONSGIV         int64
VEHSRCHCOND        int64
VEHSRCHCONDBY      int64
VEHCONTRA          int64
VEHDRUGS           int64
VEHPARA            int64
VEHALC             int64
VEHWEAP            int64
VEHSTOLPROP        int64
VEHOTHER           int64
VEHDRAMT           int64
dtype: object

In [ ]:
search.iloc[:,0]

0          1
1          1
2          1
3          1
4          1
          ..
377893    12
377894    12
377895    12
377897    12
377898    12
Name: MONTH, Length: 289780, dtype: int64

In [ ]:
search_label = {0: "Not applied", 1: "Yes", 2: "No"}
drug_amount = {0: "Not applied", 1: "< 2 grams",
               2: "2-10 grams", 3: "11-50 grams", 4: "51-100 grams", 
               5: "> 100 grams"} 

In [ ]:
for i in range(6,17):
  search.iloc[:,i] = search.iloc[:,i].map(search_label)
search.iloc[:,-1] = search.iloc[:,-1].map(drug_amount)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [ ]:
search.to_csv(path+"searching_subset.csv", index=False)

### Who Gave a Search Consent?

In [ ]:
consent = search.groupby("VEHCONSREQ")["MONTH"].count().reset_index()
consent.columns = ["CONSENT", "COUNT"]
consent["PERCENT"] = round(consent.COUNT/consent.COUNT.sum(), 4)*100
consent

,CONSENT,COUNT,PERCENT
0,No,287269,99.13
1,Yes,2511,0.87


In [ ]:
base = alt.Chart(consent, title = "Share of Drivers in Traffic Stops Giving a Search Consent").encode(
    theta=alt.Theta("COUNT", stack=True), color=alt.Color("CONSENT", legend=None)
)

pie = base.mark_arc(outerRadius=120)
text = base.mark_text(radius=140, size=20).encode(text="CONSENT")

pie + text


alt.LayerChart(...)

In [ ]:
search_yes = search[search.VEHCONSREQ == 'Yes']

In [ ]:
consent_race = search_yes.groupby("DRRACE_NEW")["MONTH"].count().reset_index()
consent_race.columns = ["RACE", "COUNT"]
consent_race["PERCENT"] = round(consent_race.COUNT/consent_race.COUNT.sum(), 4)*100
consent_race

,RACE,COUNT,PERCENT
0,Black or African American,1946,77.50
1,Hispanic or Latino,511,20.35
2,Other,16,0.64
3,White,38,1.51


In [ ]:
consent_race.dtypes

RACE        object
COUNT        int64
PERCENT    float64
dtype: object

In [ ]:
alt.Chart(consent_race, title = "Share of Drivers by Race who Gave a Search Consent").mark_arc(outerRadius=80).encode(
    alt.Theta('COUNT'),
    alt.Color('RACE',
        scale=alt.Scale(domain=list(consent_race.RACE.unique())),
        legend=alt.Legend(title=None, orient='none', legendX=260, legendY=50)),
    order='COUNT'
).properties(width=300, height=200).configure_view(strokeOpacity=0)

In [ ]:
alt.Chart(consent_race, title = "Share of Drivers by Race who Gave a Search Consent").mark_arc(outerRadius=80).encode(
    alt.Theta('COUNT'),
    alt.Color('RACE',
        scale=alt.Scale(domain=list(consent_race.RACE.unique())),
        legend=alt.Legend(title=None, orient='none', legendX=260, legendY=50)),
    order='COUNT'
).properties(width=300, height=200).configure_view(strokeOpacity=0)

alt.Chart(...)

In [ ]:
consent_race_all = search.groupby("DRRACE_NEW")["MONTH"].count().reset_index()
consent_race_all.columns = ["RACE", "COUNT"]
consent_race_all["PERCENT"] = round(consent_race_all.COUNT/consent_race_all.COUNT.sum(), 4)*100
consent_race_all["CONSENT"] = ["Both"]*4

consent_race["CONSENT"] = ["Yes only"]*4

In [ ]:
consent_comp = pd.concat([consent_race_all, consent_race]).reset_index().iloc[:,1:]

In [ ]:
alt.Chart(consent_comp, title  = "Who Gave Consent vs Who was Searched").mark_bar().encode(
    x= alt.X('CONSENT', sort = ["Both", "Yes only"], title  = "", axis = None),
    y=alt.Y('PERCENT', title  = ""),
    column = alt.Column('RACE', spacing = 5, header = alt.Header(labelOrient = "top"), title=""),
    color=alt.Color('CONSENT', legend=alt.Legend(orient='none', title = 'Consent Given?', direction = "horizontal", legendX = 250, legendY = 320)),
).configure_view(stroke='transparent').properties(
    width = 150, 
    title=alt.TitleParams(     # This works but at the cost of sacrificing the title
        ['Who Gave Consent vs Who was Searched'],
        baseline='top',
        orient='top',
        anchor='middle',
        fontWeight= 600,
       fontSize=12))

alt.Chart(...)

### What was Found from the Search?

* Drug

In [579]:
drug_if = search.groupby("VEHDRUGS")["MONTH"].count().reset_index()
drug_if.columns = ["DRUG", "COUNT"]
drug_if = drug_if[drug_if.DRUG.isin(["Yes", "No"])].reset_index().iloc[:,1:]
drug_if.loc[:, "PERCENT"] = round(drug_if.COUNT/drug_if.COUNT.sum(),4)*100
drug_if.loc[:, "Contraband"] = ["Drugs"]*2
drug_if

,DRUG,COUNT,PERCENT,Contraband
0,No,3516,78.5,Drugs
1,Yes,963,21.5,Drugs


In [464]:
base_drug = alt.Chart(drug_if, title = "Proportion of Traffic Stops that Found Drugs").encode(
    theta=alt.Theta("COUNT", stack=True), color=alt.Color("DRUG", legend=None)
)

pie_drug = base_drug.mark_arc(outerRadius=100)
text_drug = base_drug.mark_text(radius=140, size=16).encode(text="DRUG")

pie_drug + text_drug + base_drug.mark_text(radius=120, size=16).encode(text="COUNT")

alt.LayerChart(...)

* Drug Paraphernalia

In [578]:
drugPara = search.groupby("VEHPARA")["MONTH"].count().reset_index()
drugPara.columns = ["DRUG", "COUNT"]
drugPara = drugPara[drugPara.DRUG.isin(["Yes", "No"])].reset_index().iloc[:,1:]
drugPara.loc[:, "PERCENT"] = round(drugPara.COUNT/drugPara.COUNT.sum(),4)*100
drugPara.loc[:, "Contraband"] = ["Drug Paraphernalia"]*2

drugPara

,DRUG,COUNT,PERCENT,Contraband
0,No,4453,99.42,Drug Paraphernalia
1,Yes,26,0.58,Drug Paraphernalia


* Alcohol

In [581]:
alcohol = search.groupby("VEHALC")["MONTH"].count().reset_index()
alcohol.columns = ["DRUG", "COUNT"]
alcohol = alcohol[alcohol.DRUG.isin(["Yes", "No"])].reset_index().iloc[:,1:]
alcohol.loc[:, "PERCENT"] = round(alcohol.COUNT/alcohol.COUNT.sum(),4)*100
alcohol.loc[:, "Contraband"] = ["Alcohol"]*2

alcohol

,DRUG,COUNT,PERCENT,Contraband
0,No,4227,94.37,Alcohol
1,Yes,252,5.63,Alcohol


* Weapon

In [583]:
weapon = search.groupby("VEHWEAP")["MONTH"].count().reset_index()
weapon.columns = ["DRUG", "COUNT"]
weapon = weapon[weapon.DRUG.isin(["Yes", "No"])].reset_index().iloc[:,1:]
weapon.loc[:, "PERCENT"] = round(weapon.COUNT/weapon.COUNT.sum(),4)*100
weapon.loc[:, "Contraband"] = ["Weapon"]*2

weapon

,DRUG,COUNT,PERCENT,Contraband
0,No,4289,95.76,Weapon
1,Yes,190,4.24,Weapon


* Stolen Property

In [585]:
#VEHSTOLPROP 
#VEHOTHER

prop = search.groupby("VEHSTOLPROP")["MONTH"].count().reset_index()
prop.columns = ["DRUG", "COUNT"]
prop = prop[prop.DRUG.isin(["Yes", "No"])].reset_index().iloc[:,1:]
prop.loc[:, "PERCENT"] = round(prop.COUNT/prop.COUNT.sum(),4)*100
prop.loc[:, "Contraband"] = ["Stolen property"]*2

prop

,DRUG,COUNT,PERCENT,Contraband
0,No,4477,99.96,Stolen property
1,Yes,2,0.04,Stolen property


* Other

In [588]:
other = search.groupby("VEHOTHER")["MONTH"].count().reset_index()
other.columns = ["DRUG", "COUNT"]
other = other[other.DRUG.isin(["Yes", "No"])].reset_index().iloc[:,1:]
other.loc[:, "PERCENT"] = round(other.COUNT/other.COUNT.sum(),4)*100
other.loc[:, "Contraband"] = ["Other"]*2

other

,DRUG,COUNT,PERCENT,Contraband
0,No,4428,98.86,Other
1,Yes,51,1.14,Other


* Comparison

In [624]:
contrabands = pd.concat([drug_if, drugPara, alcohol, weapon, prop, other]).reset_index()
contrabands = contrabands.iloc[:,1:]

In [625]:
for r in range(contrabands.shape[0]):
  if contrabands.loc[r, "DRUG"] == "No":
    contrabands.loc[r, "DRUG"] = "Not found contrabands"
  elif contrabands.loc[r, "DRUG"] == "Yes":
    contrabands.loc[r, "DRUG"] = "Found contrabands"

In [626]:
contrabands

,DRUG,COUNT,PERCENT,Contraband
0,Not found contrabands,3516,78.50,Drugs
1,Found contrabands,963,21.50,Drugs
2,Not found contrabands,4453,99.42,Drug Paraphernalia
3,Found contrabands,26,0.58,Drug Paraphernalia
4,Not found contrabands,4227,94.37,Alcohol
5,Found contrabands,252,5.63,Alcohol
6,Not found contrabands,4289,95.76,Weapon
7,Found contrabands,190,4.24,Weapon
8,Not found contrabands,4477,99.96,Stolen property
9,Found contrabands,2,0.04,Stolen property


In [630]:
contraband_num = contrabands[contrabands.DRUG == "Found contrabands"].reset_index().iloc[:,[-1,-3,-2]]
contraband_num

,Contraband,COUNT,PERCENT
0,Drugs,963,21.50
1,Drug Paraphernalia,26,0.58
2,Alcohol,252,5.63
3,Weapon,190,4.24
4,Stolen property,2,0.04
5,Other,51,1.14


In [639]:
alt.Chart(contraband_num, title = "Type of Contrabands, From the Most to the Least").mark_bar().encode(
    x= alt.X('COUNT', axis=alt.Axis(title='', domain = False, ticks=False, tickCount=5, grid=True)),
    y=alt.Y('Contraband', sort= '-x', 
            axis=alt.Axis(title='', 
            titleFontSize = 14, titleFontWeight = 600, titleAngle = 0,
            titleY = -5, titleX = -10, titlePadding= 10,ticks=False, domain=False))
    ).configure_view(
      strokeWidth=0
  ).properties(width = 400, height = 300)

alt.Chart(...)

In [623]:
alt.Chart(contrabands, title="Proportion of Traffic Searches that Found a Contraband").mark_bar().encode(
    y= alt.Y('Contraband', title='', sort = "-x", axis=alt.Axis(title='', 
            titleFontSize = 12, titleFontWeight = 600, titleAngle = 0,
            titleY = -5, titleX = -10, titlePadding= 10,ticks=False, domain=False)),
    x=alt.X('COUNT',stack="normalize", title='', axis=alt.Axis(format='%', ticks=False, domain=False)),
    color= alt.Color('DRUG', legend=alt.Legend(orient='none', title = '', direction = "horizontal", legendX = 120, legendY = 430, labelFontSize=15),
                     scale=alt.Scale(domain=['Not found contrabands', 'Found contrabands'], 
                                     scheme='purplegreen')),
    order=alt.Order(
      'DRUG',
      sort='descending'
    )
).configure_axis(
      grid=False
  ).configure_view(
      strokeWidth=0
  ).properties(width = 600, height = 400)

alt.Chart(...)

### How Many Drugs Were Seized in Traffic Stops?

* Overall

In [499]:
drug = search.groupby("VEHDRAMT")["MONTH"].count().reset_index().iloc[:-1,:]
drug.columns = ["AMOUNT", "COUNT"]
drug["PERCENT"] = round(drug.COUNT/drug.COUNT.sum(), 4)*100
drug = drug.iloc[[3,1,0,2,4],:]
drug

,AMOUNT,COUNT,PERCENT
3,< 2 grams,340,35.31
1,2-10 grams,451,46.83
0,11-50 grams,121,12.56
2,51-100 grams,25,2.60
4,> 100 grams,26,2.70


In [454]:
alt.Chart(drug, title = "Amount of Drugs Seized in Traffic Stops").mark_bar().encode(
      x= alt.X('AMOUNT', title = "" , sort = list(drug.AMOUNT.unique()), axis=alt.Axis(labelAngle=-45, domain = False, ticks = False)),
      y= alt.Y('COUNT', title = "", axis=alt.Axis(tickCount = 7, ticks = False, domain=False))
  ).configure_axis(
      grid=False
  ).configure_view(
      strokeWidth=0
  ).properties(width = 400, height = 400)

alt.Chart(...)

* Inside Eace Racial Group

In [ ]:
drug_race = search[search["VEHDRAMT"] != "Not applied"].groupby(["DRRACE_NEW", "VEHDRAMT"])["MONTH"].count().unstack(fill_value=0).stack().reset_index()
drug_race.columns = ["RACE", "AMOUNT", "COUNT"]
drug_race["PERCENT"] = round(100 * drug_race['COUNT'] / drug_race.groupby('RACE')['COUNT'].transform('sum'), 2)  # https://stackoverflow.com/questions/23377108/pandas-percentage-of-total-with-groupby
drug_race

In [510]:
c_size = 200

chart_black = alt.Chart(drug_race.iloc[:5,:], title = "Black or African American").mark_bar().encode(
      x= alt.X('AMOUNT', title = "" , sort = list(drug.AMOUNT.unique()), axis=alt.Axis(labelAngle=-45, ticks = False)),
      y= alt.Y('COUNT', title = "", axis=alt.Axis(tickCount = 7, ticks = False, domain=False))
  ).properties(width = c_size, height = c_size)

chart_white = alt.Chart(drug_race.iloc[15:,:], title = "White").mark_bar().encode(
      x= alt.X('AMOUNT', title = "" , sort = list(drug.AMOUNT.unique()), axis=alt.Axis(labelAngle=-45,  ticks = False)),
      y= alt.Y('COUNT', title = "", axis=alt.Axis(tickCount = 7, ticks = False, domain=False))
  ).properties(width = c_size, height = c_size)

chart_hispanic = alt.Chart(drug_race.iloc[5:10,:], title = "Hispanic or Latino").mark_bar().encode(
      x= alt.X('AMOUNT', title = "" , sort = list(drug.AMOUNT.unique()), axis=alt.Axis(labelAngle=-45, ticks = False)),
      y= alt.Y('COUNT', title = "", axis=alt.Axis(tickCount = 7, ticks = False, domain=False))
  ).properties(width = c_size, height = c_size)

chart_other = alt.Chart(drug_race.iloc[10:15,:], title = "Other Race").mark_bar().encode(
      x= alt.X('AMOUNT', title = "" , sort = list(drug.AMOUNT.unique()), axis=alt.Axis(labelAngle=-45, ticks = False)),
      y= alt.Y('COUNT', title = "", axis=alt.Axis(tickCount = 7, ticks = False, domain=False))
  ).properties(width = c_size, height = c_size)

chart_allrace = (chart_white | chart_black) & (chart_hispanic | chart_other)

chart_allrace.configure_axis(
      grid=False
  ).configure_view(
      strokeWidth=0
  )

alt.VConcatChart(...)

* Comparison

In [515]:
drug_r = search.groupby(["DRRACE_NEW", "VEHDRUGS"])["MONTH"].count().reset_index()
drug_r.columns = ["RACE","DRUG", "COUNT"]
drug_r = drug_r[drug_r.DRUG.isin(["Yes", "No"])].reset_index().iloc[:,1:]
drug_r

,RACE,DRUG,COUNT
0,Black or African American,No,2786
1,Black or African American,Yes,731
2,Hispanic or Latino,No,662
3,Hispanic or Latino,Yes,203
4,Other,No,14
5,Other,Yes,9
6,White,No,54
7,White,Yes,20


In [553]:
drug_total_lst = []
for race in list(drug_r.RACE.unique()):
  total_num = drug_r[drug_r.RACE == race].COUNT.sum()
  drug_total_lst.append(total_num)
drug_total_lst

drug_racecompare = pd.concat([drug_r, 
           pd.DataFrame({"RACE":drug_r.RACE.unique(), "DRUG": ["All drug searches"]*4, "COUNT":drug_total_lst})])
drug_racecompare = drug_racecompare[drug_racecompare.DRUG != "No"].reset_index()
drug_racecompare = drug_racecompare.iloc[[3,7,0,4,1,5,2,6],:].reset_index().iloc[:,2:]
drug_racecompare

,RACE,DRUG,COUNT
0,White,Yes,20
1,White,All drug searches,74
2,Black or African American,Yes,731
3,Black or African American,All drug searches,3517
4,Hispanic or Latino,Yes,203
5,Hispanic or Latino,All drug searches,865
6,Other,Yes,9
7,Other,All drug searches,23


In [560]:
for i in range(drug_racecompare.shape[0]):
  if drug_racecompare.loc[i, "DRUG"] == "Yes":
    drug_racecompare.loc[i, "DRUG"] = "Searches that found drugs"
drug_racecompare

,RACE,DRUG,COUNT
0,White,Searches that found drugs,20
1,White,All drug searches,74
2,Black or African American,Searches that found drugs,731
3,Black or African American,All drug searches,3517
4,Hispanic or Latino,Searches that found drugs,203
5,Hispanic or Latino,All drug searches,865
6,Other,Searches that found drugs,9
7,Other,All drug searches,23


In [574]:
alt.Chart(drug_racecompare, title = "Drug Search Results by Race").mark_bar(opacity=0.8).encode(
    x= alt.X('RACE', title = "", axis=alt.Axis(ticks=False, labelAngle= 0, domain =False)),
    y=alt.Y('COUNT', stack=None, title = "", axis=alt.Axis(ticks = False, domain = False)),
    color=alt.Color('DRUG', legend=alt.Legend(orient='none', title = '', direction = "horizontal", legendX = 90, legendY = 345),
                    scale=alt.Scale(range=['#ddccbb', 'red']))
).configure_view(
      strokeWidth=0
  ).properties(width = 450, height = 320)

alt.Chart(...)